# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [1]:
var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var writeFileSync = require('fs').writeFileSync;

// add some run commands to the bash script
var bashToRun = (code) => code.split('\n').map(l => 'RUN ' + l)
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');

// create a selenium Dockerfile with a vnc connection
var interpretAllPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');
var seleniumDocker = (outputFile) => {
    return importer.importNotebook(interpretAllPath)
        .then((f) => {
            return Promise.all([
            interpret('run selenium'),
            // add some extra services
            interpret('linux dev tools'),
            interpret('vnc html'),
            interpret('vnc docker')
        ])})
        .then(r => r.map(r => r[0]))
        .then(r => interpretObject(r))
        .then(r => {
            // lets fix the middle result
            r[1].code = bashToRun(r[1].code);
            r[2].code = bashToRun(r[2].code);
            return r.map(r => r.code).join('\n');
        })
        .then(r => {
            // save the Dockerfile
            writeFileSync(outputFile, r);
            return r;
        });
};
(seleniumDocker);


[Function: seleniumDocker]

How to set up selenium server?

In [2]:
var path = require('path');
var execSync = require('child_process').execSync;
var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');
var getSeleniumServer = () => {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    return seleniumDocker(path.join(__dirname, 'Dockerfile'))
        .then(r => execSync('docker build -t act-selenium .').toString())
        .then(r => {
            var ps = execSync('docker ps -a').toString();
            if(ps.indexOf('act-selenium') !== -1) {
                return r 
                    + execSync('docker stop act-selenium').toString()
                    + execSync('docker rm act-selenium').toString();
            }
            return '';
        })
        .then(r => r + execSync('docker run --user=seluser --shm-size=2g --name act-selenium -d -p 8888:8888 -p 6080:6080 -p 5900:5900 -p 4444:4444 -p 4200:4200 -p 3000:3000 -v "' + PROFILE_DIR + '":/usr/profile -v "' + DOWNLOAD_DIR + '":/data/downloads act-selenium').toString());
};
(getSeleniumServer);


[Function: getSeleniumServer]

In [3]:
$$.async();
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(r => $$.sendError(r));

'Sending build context to Docker daemon  54.78kB\r\r\nStep 1/15 : FROM selenium/standalone-chrome-debug\n ---> 870685c556e8\nStep 2/15 : EXPOSE 4200\n ---> Using cache\n ---> da859600ae82\nStep 3/15 : EXPOSE 4444\n ---> Using cache\n ---> 51ab31a51459\nStep 4/15 : EXPOSE 3000\n ---> Using cache\n ---> de8866927da3\nStep 5/15 : ENV CHROME_USER_DATA_DIR /usr/profile\n ---> Using cache\n ---> 3d01edce8e39\nStep 6/15 : WORKDIR /home/seluser\n ---> Using cache\n ---> a80023c69bc3\nStep 7/15 : USER root\n ---> Using cache\n ---> 579b9aa44ccd\nStep 8/15 : VOLUME /usr/profile\n ---> Using cache\n ---> 850a03bab611\nStep 9/15 : VOLUME /data/downloads\n ---> Using cache\n ---> 8a67833fdeff\nStep 10/15 : RUN mkdir /var/lib/apt/lists/partial\n ---> Using cache\n ---> ed475371f67a\nStep 11/15 : RUN apt-get -qq update\n ---> Using cache\n ---> 14381af16b55\nStep 12/15 : RUN apt-get install -y --fix-missing git curl wget zip unzip vim dos2unix g++ python net-tools make websockify novnc\n ---> Using c

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true) in a separate browser


In [6]:
var vncIframe = () => `
<div style="display:block; width:100%; padding-bottom:75%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="http://localhost:6080/vnc.html?password=secret&amp;host=localhost&amp;port=6080&amp;autoconnect=true&amp;resize=downscale&amp;view_only=true"></iframe></div>
`;
(vncIframe);

In [8]:
$$.mime({'text/html': vncIframe()});

## 3

How to run a selenium test on the Docker machine?
